In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import cv2
import numpy as np
import math
from scipy.stats.stats import pearsonr
from skimage.metrics import peak_signal_noise_ratio
import csv
from matplotlib.ticker import MultipleLocator
from pathlib import Path


In [2]:
def append_list_as_row(csv_file,num_case, f_num, value):
    with open(csv_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerows([num_case])
        writer.writerows([f_num])
        writer.writerows([value])


In [3]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [4]:
def normalize(y_true, y_pred):
    # cast the target images to integer
    y_true = y_true * 255.0
    y_true = y_true.astype(np.uint8)
    y_true = np.clip(y_true, 0, 255)
    
    # cast the predicted images to integer
    y_pred = y_pred * 255.0
    y_pred = y_pred.astype(np.uint8)
    y_pred = np.clip(y_pred, 0, 255)
    # return the psnr
    return y_true, y_pred

In [5]:
os.chdir(
    '/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Full_wavefield_frames_time_series_project/Datasets/label_set')
f_start = np.load('frames_initial.npy')
print(f_start[396])
print(f_start[437])
print(f_start[447])
print(f_start[455])

127
154
159
211


In [6]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Sequence_to_sequence/Dataset_Full_wavefield_outputs_bottom')
c = np.load('labels_centers.npy')

In [7]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/num_csv_files_PSNR_Pearson')
psnr_damage = 'PSNR_at_delamination_area.csv'
psnr_full = 'PSNR_full_area.csv'
pearson_damage = 'Pearson_at_delamination_area.csv'
pearson_full = 'Pearson_full_area.csv'

# Enter case num

In [8]:
caselist = [397, 438, 448, 456]
for case in caselist:
    print(c[case-1])
    PSNR = []
    Pearson_CC = []
    frames = []
    print('case :', case)
    for f in range(1,513):        
        frames.append(f)              
        
        ############################################################################################################
        ## Numerical test cases
        ############################################################################################################
        os.chdir('/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset2_bottom_out/%d_output/' %case)
        ref_data = cv2.imread('%d_flat_shell_Vz_%d_500x500bottom.png' %(f, case), 0)        
        ref_data = cv2.resize(ref_data, (512,512), interpolation=cv2.INTER_CUBIC)
        os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Numerical_results_Full_wavefield/%d_output/' %case)
        pred_data = cv2.imread('Ijjeh_SR_Pred_output_%d_frame_output_%d.png' %(f, case), 0)
        ############################################################################################################
        
        ref_img = np.asarray(ref_data, dtype=np.uint8)         
        pred_img = np.asarray(pred_data, dtype=np.uint8)
        
        # # Comment / Uncomment to generate the damage area
        # if c[case-1][0]<=50 and c[case-1][1]<=50:
        #     ref_img = ref_img[0:100, 0:100]
        #     pred_img = pred_img[0:100, 0:100]
        # elif c[case-1][0]<=50 and c[case-1][1]>50:
        #     ref_img = ref_img[0:100, c[case-1][1]-50:c[case-1][1]+50]
        #     pred_img = pred_img[0:100, c[case-1][1]-50:c[case-1][1]+50]
        # elif c[case-1][0]>50 and c[case-1][1]<=50:
        #     ref_img = ref_img[c[case-1][0]-50:c[case-1][0]+50, 0:100]
        #     pred_img = pred_img[c[case-1][0]-50:c[case-1][0]+50, 0:100]
        # else:
        #     ref_img = ref_img[c[case-1][0]-50:c[case-1][0]+50, c[case-1][1]-50:c[case-1][1]+50]
        #     pred_img = pred_img[c[case-1][0]-50:c[case-1][0]+50, c[case-1][1]-50:c[case-1][1]+50  
        # 
        # ref_img = ref_img - np.mean(ref_img)
        # pred_img = pred_img - np.mean(pred_img)
                                
        temp_pearson = np.corrcoef(ref_img.flat, pred_img.flat)
        Pearson_CC.append(temp_pearson[0][1])
        
        # ref_img, pred_img = normalize(ref_img, pred_img)
        
        
        temp_psnr = peak_signal_noise_ratio(ref_img, pred_img, data_range=255)
        PSNR.append(temp_psnr)
    
        
        fig = plt.figure(figsize=(1, 1), dpi=512)  
        ax = fig.add_subplot(111)
        
        # Select what to show  ref_img / pred_img
        ax.imshow(ref_img)  
        ax.axis('off')
        # plt.show()
        
        # Comment / Uncomment to generate full frames of localised damage
        if c[case-1][0]<=50 and c[case-1][1]<=50:
            ax.add_patch(patches.Rectangle((0, 0), 100, 100, linewidth=0.125, edgecolor='k', facecolor='none'))             
        elif c[case-1][0]<=50 and c[case-1][1]>50:
            ax.add_patch(patches.Rectangle((c[case-1][1]-50, 0), 100, 100, linewidth=0.125, edgecolor='k', facecolor='none')) 
        elif c[case-1][0]>50 and c[case-1][1]<=50:
            ax.add_patch(patches.Rectangle((0, c[case-1][0]-50), 100, 100, linewidth=0.125, edgecolor='k', facecolor='none')) 
        else:
            ax.add_patch(patches.Rectangle((c[case-1][1]-50, c[case-1][0]-50), 100, 100, linewidth=0.125, edgecolor='k', facecolor='none')) 
               
        
        
        ##########################################################################################################
        path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/num_delamination_full_frame//%d_output/' % case
        Path(path).mkdir(parents=True, exist_ok=True)
        os.chdir(path)
        fig.savefig('output_%d_frame_%d_full_frame_GT.png' %(case, f),bbox_inches='tight', transparent="True", pad_inches=0)
        plt.close('all')
    
    ####################################################################################################
                                
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/num_csv_files_PSNR_Pearson/')
    # append_list_as_row(psnr_full, [case],frames, PSNR)
    # append_list_as_row(pearson_full, [case],frames, Pearson_CC)
    ##################################################################################################
    
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/num_PSNR_Pearson_figs/')    
    plt.figure(figsize=(20/2.53,10/2.53), dpi=600)
    plt.plot(Pearson_CC)
    plt.xlabel('frame')
    plt.ylabel('Pearson CC')
    plt.savefig("Ijjeh_Pearson_CC_num_case_%d_512_frames_full.png" % case)
    ##################################################################################################
    
    plt.figure(figsize=(20/2.53,10/2.53), dpi=600)
    plt.plot(PSNR)
    plt.xlabel('frame')
    plt.ylabel('PSNR')
    plt.savefig("Ijjeh_PSNR_num_case_%d_512_frames_full.png" % case)
    plt.close('all')


[104. 112.]
case : 397
[  4. 226.]
case : 438
[181.   4.]
case : 448
[ 5. 14.]
case : 456
